In [6]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from collections import Counter
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from PIL import Image, ImageFilter
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from keras import models, regularizers, layers, optimizers, losses, metrics
from keras.models import Sequential
from keras.layers import Dense
from keras.preprocessing import image
from keras.applications import ResNet50

In [7]:

# Adjust the path to match your dataset
data_dir = '/kaggle/input/cv25-project-dataset/Project Data/Train/images'

# Explore the directory structure
for dirname, _, filenames in os.walk(data_dir):
    print(f"Directory: {dirname}")
    for filename in filenames:
        print(f" - {filename}")


Directory: /kaggle/input/cv25-project-dataset/Project Data/Train/images
Directory: /kaggle/input/cv25-project-dataset/Project Data/Train/images/Tomato___Late_blight
 - 766771e1-8e8e-4ce4-9af6-b4ed91c9e80a___RS_Late.B 6881.JPG
 - 014d4972-1301-46d3-afc4-56470b774137___GHLB2 Leaf 8831.JPG
 - f8024ad7-ded1-42aa-ab6d-139f7069d10e___RS_Late.B 7157.JPG
 - f5261645-e84e-433a-8edb-ff53e8586646___GHLB2 Leaf 143.1.JPG
 - 017a9839-6097-45aa-85b0-3051db151484___RS_Late.B 5125.JPG
 - 2ac8376a-3401-4572-af45-6984e6415996___GHLB2 Leaf 8787.JPG
 - e1d00dcb-f2cb-41ca-8a45-a46c43e7c90e___GHLB2 Leaf 8697.JPG
 - 16c1763a-8e49-436c-865e-be56d8dadf4f___GHLB2 Leaf 9107.JPG
 - ca6362b2-b680-4348-a795-3bc37275e70f___GHLB2 Leaf 79.JPG
 - 0e244a32-08cd-471c-90dd-141c354c20b3___GHLB2 Leaf 8634.JPG
 - 8daa8725-05a2-4802-a524-9eba96fa89bc___RS_Late.B 6488.JPG
 - 2d583f2c-3e59-4c4c-be45-5f367a38af95___GHLB2 Leaf 8724.JPG
 - 63385691-5de8-4652-abe5-15727fdcab4b___RS_Late.B 5342.JPG
 - 06b911b8-1375-4da1-929c-2294c405

In [8]:
# Adjust the path to match your dataset
data_dir = '/kaggle/input/cv25-project-dataset/Project Data/Train/'

# Explore the directory structure
for dirname, _, filenames in os.walk(data_dir):
    print(f"Directory: {dirname}")


Directory: /kaggle/input/cv25-project-dataset/Project Data/Train/
Directory: /kaggle/input/cv25-project-dataset/Project Data/Train/images
Directory: /kaggle/input/cv25-project-dataset/Project Data/Train/images/Tomato___Late_blight
Directory: /kaggle/input/cv25-project-dataset/Project Data/Train/images/Tomato___healthy
Directory: /kaggle/input/cv25-project-dataset/Project Data/Train/images/Grape___healthy
Directory: /kaggle/input/cv25-project-dataset/Project Data/Train/images/Potato___healthy
Directory: /kaggle/input/cv25-project-dataset/Project Data/Train/images/Corn_(maize)___Northern_Leaf_Blight
Directory: /kaggle/input/cv25-project-dataset/Project Data/Train/images/Tomato___Early_blight
Directory: /kaggle/input/cv25-project-dataset/Project Data/Train/images/Tomato___Septoria_leaf_spot
Directory: /kaggle/input/cv25-project-dataset/Project Data/Train/images/Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot
Directory: /kaggle/input/cv25-project-dataset/Project Data/Train/images/Strawb

In [43]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset, random_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from PIL import Image

# Define constants
image_size = (224, 224)
train_dir = '/kaggle/input/cv25-project-dataset/Project Data/Train/images'
test_dir = '/kaggle/input/cv25-project-dataset/Project Data/Val/images'  # Treat val as test
batch_size = 64
validation_split = 0.2  # 20% of training data for validation

# Function to load data
def load_data(data_dir):
    """Load images and labels from the directory."""
    images = []
    labels = []
    for label in os.listdir(data_dir):
        class_dir = os.path.join(data_dir, label)
        if os.path.isdir(class_dir):
            main_label = label.split("___")[0]  # Extract main label
            for img_name in os.listdir(class_dir):
                img_path = os.path.join(class_dir, img_name)
                if os.path.isfile(img_path):
                    try:
                        img = Image.open(img_path).convert('RGB')
                        img = img.resize(image_size)
                        img_array = np.array(img)
                        images.append(img_array)
                        labels.append(main_label)
                    except Exception as e:
                        print(f"Error loading {img_path}: {e}")
    return np.array(images), np.array(labels)

# Load data
train_images, train_labels = load_data(train_dir)
test_images, test_labels = load_data(test_dir)

# Normalize images to [0, 1]
train_images = train_images / 255.0
test_images = test_images / 255.0

# Instantiate the encoder
label_encoder = LabelEncoder()

# Fit the encoder on the label data and transform
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Retrieve mappings
id2label = {idx: label for idx, label in enumerate(label_encoder.classes_)}
label2id = {label: idx for idx, label in id2label.items()}

num_classes = len(label_encoder.classes_)

# Compute class weights for imbalanced data
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_labels_encoded),
    y=train_labels_encoded
)
class_weights = dict(enumerate(class_weights))

# Split the training data into training and validation
train_size = int((1 - validation_split) * len(train_images))
val_size = len(train_images) - train_size
train_images, val_images = train_images[:train_size], train_images[train_size:]
train_labels_encoded, val_labels_encoded = train_labels_encoded[:train_size], train_labels_encoded[train_size:]

# Custom Dataset to wrap image and label arrays
class CustomImageDataset(Dataset):
    def __init__(self, images, labels):
        self.images = torch.tensor(images, dtype=torch.float32).permute(0, 3, 1, 2)  # Convert HWC to CHW
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {"pixel_values": self.images[idx], "labels": self.labels[idx]}

# Wrap training, validation, and test data
train_dataset = CustomImageDataset(train_images, train_labels_encoded)
val_dataset = CustomImageDataset(val_images, val_labels_encoded)
test_dataset = CustomImageDataset(test_images, test_labels_encoded)

# Now you can use `train_dataset`, `val_dataset`, and `test_dataset` for training, validation, and testing


In [25]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam

# ViT

In [34]:
!pip uninstall keras
!pip install tf-keras

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Found existing installation: keras 2.15.0
Uninstalling keras-2.15.0:
  Would remove:
    /opt/conda/lib/python3.10/site-packages/keras-2.15.0.dist-info/*
    /opt/conda/lib/python3.10/site-packages/keras/*
Proceed (Y/n)? ^C
ERROR: Operation cancelled by user
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 MB 2.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 75.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 97.1 MB/s eta 0:00:00:00:01
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.15.2
    Uninstalling tensorboard-2.15.2:
      Successfully uninstalled tensorboard-2.15.2
  Attempting uninstall: keras
    Found existing installation: keras 2.15.

In [17]:
!pip install transformers

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [27]:
from transformers import ViTForImageClassification

In [47]:
from transformers import TrainingArguments, Trainer
import numpy as np

train_args = TrainingArguments(
    output_dir="output-models",
    save_total_limit=2,
    save_strategy="epoch",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=10,  # Adjust epochs as needed
    weight_decay=0.01,
    logging_dir="logs",
    logging_steps=10,
    load_best_model_at_end=True,
    report_to="none",  # Avoid W&B if unnecessary
    remove_unused_columns=False,
)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [44]:

# Load the ViT model from Hugging Face Transformers
model_name = "google/vit-base-patch16-224"  # Replace with your choice
model = ViTForImageClassification.from_pretrained(
    model_name,
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True
)

# Model loaded successfully
print(f"Model {model_name} loaded with {num_classes} classes.")

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([9]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([9, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model google/vit-base-patch16-224 loaded with 9 classes.


In [46]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    accuracy = (predictions == labels).mean()
    return {"accuracy": accuracy}


In [48]:
from torch.nn import CrossEntropyLoss
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        """
        Custom loss computation with class weights.
        """
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits

        # Define CrossEntropyLoss with class weights
        loss_fn = CrossEntropyLoss(weight=torch.tensor(list(class_weights.values()), dtype=torch.float32).to(logits.device))
        loss = loss_fn(logits, labels)

        # Return loss and model outputs if needed
        return (loss, outputs) if return_outputs else loss


In [49]:
trainer = CustomTrainer(
    model=model,
    args=train_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=None,
    compute_metrics=compute_metrics,
)

/tmp/ipykernel_23/3370262070.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CustomTrainer.__init__`. Use `processing_class` instead.
  trainer = CustomTrainer(


In [50]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.466600,0.664644,0.909091
2,0.036300,0.233334,0.958333
3,0.010300,0.202557,0.958333
4,0.004300,0.164476,0.969697
5,0.003100,0.151068,0.969697
6,0.002600,0.147092,0.973485
7,0.002100,0.143283,0.973485
8,0.001800,0.139869,0.973485
9,0.001600,0.137955,0.973485
10,0.001700,0.136607,0.973485


TrainOutput(global_step=660, training_loss=0.1219176164785908, metrics={'train_runtime': 246.2342, 'train_samples_per_second': 42.886, 'train_steps_per_second': 2.68, 'total_flos': 8.183667504454042e+17, 'train_loss': 0.1219176164785908, 'epoch': 10.0})

In [52]:
from sklearn.metrics import recall_score, accuracy_score, f1_score, classification_report

# Use the trainer to make predictions on the test dataset
outputs = trainer.predict(test_dataset)

# Print the evaluation metrics (accuracy, loss, etc.) from the trainer
print(outputs.metrics)

# Get the predicted labels and true labels
y_true = outputs.label_ids
y_pred = outputs.predictions.argmax(axis=-1)  # Get the predicted class labels

# Calculate accuracy
accuracy = accuracy_score(y_true, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Calculate recall for each class
recall = recall_score(y_true, y_pred, average=None)

# Print recall for each class
for label, score in zip(label_encoder.classes_, recall):
    print(f"Recall for {label}: {score:.2f}")

# Calculate F1 score for each class (optional)
f1 = f1_score(y_true, y_pred, average=None)
for label, score in zip(label_encoder.classes_, f1):
    print(f"F1 Score for {label}: {score:.2f}")

# Detailed classification report (precision, recall, f1-score)
print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=label_encoder.classes_))


{'test_loss': 0.04915888234972954, 'test_accuracy': 0.9848484848484849, 'test_runtime': 2.0115, 'test_samples_per_second': 164.059, 'test_steps_per_second': 10.44}
Accuracy: 0.98
Recall for Apple: 1.00
Recall for Cherry_(including_sour): 1.00
Recall for Corn_(maize): 1.00
Recall for Grape: 0.97
Recall for Peach: 0.95
Recall for Pepper,_bell: 0.95
Recall for Potato: 1.00
Recall for Strawberry: 1.00
Recall for Tomato: 0.98
F1 Score for Apple: 0.99
F1 Score for Cherry_(including_sour): 1.00
F1 Score for Corn_(maize): 1.00
F1 Score for Grape: 0.99
F1 Score for Peach: 0.97
F1 Score for Pepper,_bell: 0.97
F1 Score for Potato: 0.95
F1 Score for Strawberry: 0.98
F1 Score for Tomato: 0.99

Classification Report:
                         precision    recall  f1-score   support

                  Apple       0.98      1.00      0.99        40
Cherry_(including_sour)       1.00      1.00      1.00        20
           Corn_(maize)       1.00      1.00      1.00        40
                  Grape   

In [53]:
trainer.save_model("final_model")